Article from Philippe:
https://practicalcheminformatics.blogspot.com/2023/06/getting-real-with-molecular-property.html


In [1]:
# load biogen data
import pandas as pd

biogen_data=pd.read_csv("ADME_public_set_3521.csv")
print(biogen_data.columns)

#load bioavailabity data
bio_avail_data = pd.read_csv("11095_2013_1222_MOESM2_ESM.csv",sep=";")
print(bio_avail_data.columns)



Index(['Internal ID', 'Vendor ID', 'SMILES', 'CollectionName',
       'LOG HLM_CLint (mL/min/kg)', 'LOG MDR1-MDCK ER (B-A/A-B)',
       'LOG SOLUBILITY PH 6.8 (ug/mL)',
       'LOG PLASMA PROTEIN BINDING (HUMAN) (% unbound)',
       'LOG PLASMA PROTEIN BINDING (RAT) (% unbound)',
       'LOG RLM_CLint (mL/min/kg)'],
      dtype='object')
Index(['No', 'Name', 'Updated SMILES', '%F', 'logK(%F)', 'Category', 'Source'], dtype='object')


use feed-forward NN using pytorch.
train on solubility from dataset 1 first.
then optimize for bioavailability

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
from deepchem.feat import RDKitDescriptors

#### DATA PREPARATION ####

# Function to generate features from SMILES strings using RDKit descriptors
def generate_features(smiles_list):
    featurizer = RDKitDescriptors()
    features = featurizer.featurize(smiles_list)
    # Drop features containing invalid values
    features = features[:, ~np.isnan(features).any(axis=0)]
    return features

#remove nan values from data
biogen_data = biogen_data.dropna(subset=['LOG SOLUBILITY PH 6.8 (ug/mL)'])

#get x and y data (x is the molecular descriptors, y is the solubility)
y_data = biogen_data["LOG SOLUBILITY PH 6.8 (ug/mL)"]

print(len(y_data))

# Generate features from SMILES data (get smiles from df)
smiles = biogen_data["SMILES"]
X_data = generate_features(smiles)

#split data into training and validation using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)
# Convert y pandas Series to NumPy array
y_train = y_train.values.astype(np.float32)
y_test = y_test.values.astype(np.float32)

#convert data to pytorch tensors (like numpy arrays but for pytorch)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Reshape the target tensor to match the shape of the output tensor
y_train_tensor = y_train_tensor.view(-1, 1)
# Reshape the target tensor to match the shape of the output tensor
y_test_tensor = y_test_tensor.view(-1, 1)


2173


In [17]:
y_train_tensor

tensor([[1.5611],
        [1.2833],
        [1.4866],
        ...,
        [1.6120],
        [1.4669],
        [1.6514]])

In [22]:
#### CREATING MODEL ####

# Define the neural network architecture
class SolubilityPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SolubilityPredictor, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Define hyperparameters
input_dim = X_data.shape[1]  # Number of molecular descriptors
hidden_dim = 64  # Adjust to get best results
output_dim = 1
learning_rate = 0.001
num_epochs = 100
batch_size = 32

#create dataloader for batch training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#initialize model, loss function (nn.MSELoss) and optimizer
model = SolubilityPredictor(input_dim, hidden_dim, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss}")


Epoch 1/100, Loss: 189.29155659675598
Epoch 2/100, Loss: 38.372320637106895
Epoch 3/100, Loss: 24.113596066832542
Epoch 4/100, Loss: 22.577118560671806
Epoch 5/100, Loss: 31.019078195095062
Epoch 6/100, Loss: 26.367550119757652
Epoch 7/100, Loss: 21.449614644050598
Epoch 8/100, Loss: 24.10814942419529
Epoch 9/100, Loss: 20.205865681171417
Epoch 10/100, Loss: 27.486654669046402
Epoch 11/100, Loss: 20.52155862748623
Epoch 12/100, Loss: 24.052323192358017
Epoch 13/100, Loss: 20.832928866147995
Epoch 14/100, Loss: 21.553574100136757
Epoch 15/100, Loss: 19.75460498034954
Epoch 16/100, Loss: 22.143404453992844
Epoch 17/100, Loss: 20.54713310301304
Epoch 18/100, Loss: 22.297346368432045
Epoch 19/100, Loss: 19.530448868870735
Epoch 20/100, Loss: 19.590066641569138
Epoch 21/100, Loss: 28.535822689533234
Epoch 22/100, Loss: 22.134691447019577
Epoch 23/100, Loss: 22.84657770395279
Epoch 24/100, Loss: 21.280014470219612
Epoch 25/100, Loss: 26.045126996934414
Epoch 26/100, Loss: 19.809996508061886


In [23]:

# Evaluation
with torch.no_grad():
    val_outputs = model(X_test_tensor)
    val_loss = criterion(val_outputs, y_test_tensor)
    print(f"Validation Loss: {val_loss.item()}")

Validation Loss: 0.44780904054641724
